# 合併患者與Calibration的語料

In [2]:
import os
import shutil
import logging

# 定義來源目錄和目標分析目錄
source_dirs = [
    "/jnrle/LDVforDysarthria/Wang_real_addnoise_LDV",
    "/jnrle/LDVforDysarthria/Wang_real_addnoise_Mic",
]

analysis_dir = "/jnrle/LDVforDysarthria/TrainingData"

# 確保分析目錄存在
os.makedirs(analysis_dir, exist_ok=True)

# 定義要合併的文字檔案
text_files = ["wav.scp", "utt2spk", "text", "spk2utt"]
# 『還要同時有wav資料夾才行』

# 定義校正用資料夾路徑
calibration_dirs = [
    "/jnrle/LDVforDysarthria/Wenen_Syn_Mic/Wenen_real_01",
]

def should_skip_file(filename):
    """檢查檔案是否應該被跳過（以._開頭）"""
    return filename.startswith('._')

# 設定日誌記錄
logging.basicConfig(filename='merge_log.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

def process_wang_dir(wang_path, source_label):
    """處理來源資料夾中的子資料夾，將符合條件的資料轉存到分析目錄中"""
    for root, dirs, files in os.walk(wang_path):
        # 檢查當前目錄是否同時包含所有四個文字檔案和wav資料夾
        if all(text_file in files for text_file in text_files) and "wav" in dirs:
            # 定義分析子資料夾名稱，包含來源標籤和相對路徑資訊
            rel_path = os.path.relpath(root, wang_path)
            # 替換路徑分隔符為下劃線以避免目錄嵌套
            rel_path_sanitized = rel_path.replace(os.sep, "_")
            subdir_name = f"{source_label}_{rel_path_sanitized}"
            subdir_path = os.path.join(analysis_dir, subdir_name)
            os.makedirs(subdir_path, exist_ok=True)
            logging.info(f"創建子資料夾: {subdir_path}")

            # 複製四個文字檔案
            for text_file in text_files:
                src_text_path = os.path.join(root, text_file)
                dst_text_path = os.path.join(subdir_path, text_file)
                if os.path.exists(src_text_path):
                    try:
                        with open(src_text_path, 'r', encoding='utf-8') as f_src:
                            content = f_src.read()
                        with open(dst_text_path, 'a', encoding='utf-8') as f_dst:
                            f_dst.write(content)
                        logging.info(f"合併文字檔案: {src_text_path} 到 {dst_text_path}")
                    except Exception as e:
                        logging.error(f"合併文字檔案失敗: {src_text_path} 到 {dst_text_path}，錯誤: {e}")

            # 複製 wav 資料夾內的所有 .wav 檔案，跳過以._開頭的檔案
            src_wav_dir = os.path.join(root, "wav")
            dst_wav_dir = os.path.join(subdir_path, "wav")
            os.makedirs(dst_wav_dir, exist_ok=True)
            logging.info(f"複製 wav 檔案從 {src_wav_dir} 到 {dst_wav_dir}")

            if os.path.exists(src_wav_dir):
                for wav_file in os.listdir(src_wav_dir):
                    if wav_file.endswith('.wav') and not should_skip_file(wav_file):
                        src_file = os.path.join(src_wav_dir, wav_file)
                        dst_file = os.path.join(dst_wav_dir, wav_file)
                        try:
                            shutil.copy(src_file, dst_file)
                            logging.info(f"複製 wav 檔案: {src_file} 到 {dst_file}")
                        except Exception as e:
                            logging.error(f"複製 wav 檔案失敗: {src_file} 到 {dst_file}，錯誤: {e}")

def merge_calibration_data():
    """將校正用資料夾的資料合併到每個分析子資料夾中"""
    for calibration_dir in calibration_dirs:
        source_label = os.path.basename(calibration_dir)
        if not os.path.exists(calibration_dir):
            logging.error(f"校正用資料夾不存在: {calibration_dir}")
            continue
        logging.info(f"處理校正用資料夾: {calibration_dir}")

        for subdir in os.listdir(analysis_dir):
            subdir_path = os.path.join(analysis_dir, subdir)
            if not os.path.isdir(subdir_path):
                continue

            # 複製 wav 資料夾內的所有 .wav 檔案，跳過以._開頭的檔案
            src_wav_dir = os.path.join(calibration_dir, "wav")
            dst_wav_dir = os.path.join(subdir_path, "wav")
            if os.path.exists(src_wav_dir):
                os.makedirs(dst_wav_dir, exist_ok=True)
                for wav_file in os.listdir(src_wav_dir):
                    if wav_file.endswith('.wav') and not should_skip_file(wav_file):
                        src_file = os.path.join(src_wav_dir, wav_file)
                        dst_file = os.path.join(dst_wav_dir, wav_file)
                        try:
                            shutil.copy(src_file, dst_file)
                            logging.info(f"複製校正 wav 檔案: {src_file} 到 {dst_file}")
                        except Exception as e:
                            logging.error(f"複製校正 wav 檔案失敗: {src_file} 到 {dst_file}，錯誤: {e}")

            # 合併文字檔，跳過以._開頭的檔案
            for text_file in text_files:
                src_text_path = os.path.join(calibration_dir, text_file)
                dst_text_path = os.path.join(subdir_path, text_file)
                if os.path.exists(src_text_path) and not should_skip_file(os.path.basename(src_text_path)):
                    try:
                        with open(src_text_path, 'r', encoding='utf-8') as f_src:
                            content = f_src.read()
                        with open(dst_text_path, 'a', encoding='utf-8') as f_dst:
                            f_dst.write(content)
                        logging.info(f"合併校正文字檔案: {src_text_path} 到 {dst_text_path}")
                    except Exception as e:
                        logging.error(f"合併校正文字檔案失敗: {src_text_path} 到 {dst_text_path}，錯誤: {e}")

# 開始合併過程
logging.info("資料合併開始")

# 處理 Wang_real_addnoise_LDV 和 Wang_real_addnoise_Mic
for wang_dir in source_dirs:
    source_label = os.path.basename(wang_dir)
    logging.info(f"處理來源資料夾: {wang_dir}，標籤: {source_label}")
    process_wang_dir(wang_dir, source_label)

# 合併校正用資料夾的資料
merge_calibration_data()

logging.info("資料合併完成！")
print("資料合併完成！")

資料合併完成！


# 修改wav.scp資料

In [3]:
import os
import shutil
import logging

# 設定日誌記錄
logging.basicConfig(
    filename='rewrite_wav_scp.log',
    level=logging.INFO,
    format='%(asctime)s:%(levelname)s:%(message)s'
)

def should_skip_file(filename):
    """檢查檔案是否應該被跳過（以._開頭）"""
    return filename.startswith('._')

def rewrite_wav_scp(analysis_dir):
    """
    重新寫入每個子資料夾中的wav.scp檔案。
    每個wav.scp將包含子資料夾內所有.wav檔案的識別符與絕對路徑。
    """
    # 獲取 TrainingData 目錄下的所有子資料夾
    for subdir in os.listdir(analysis_dir):
        subdir_path = os.path.join(analysis_dir, subdir)
        if not os.path.isdir(subdir_path):
            logging.warning(f"非目錄，跳過: {subdir_path}")
            continue

        wav_dir = os.path.join(subdir_path, "wav")
        if not os.path.exists(wav_dir):
            logging.warning(f"缺少 wav 資料夾，跳過: {wav_dir}")
            continue

        # 獲取所有 .wav 檔案，跳過以._開頭的檔案
        wav_files = [
            f for f in os.listdir(wav_dir)
            if f.endswith('.wav') and not should_skip_file(f)
        ]

        if not wav_files:
            logging.warning(f"無 .wav 檔案於: {wav_dir}")
            continue

        # 定義 wav.scp 檔案的路徑
        wav_scp_path = os.path.join(subdir_path, "wav.scp")

        try:
            with open(wav_scp_path, 'w', encoding='utf-8') as wav_scp:
                for wav_file in wav_files:
                    identifier = os.path.splitext(wav_file)[0]
                    wav_file_path = os.path.abspath(os.path.join(wav_dir, wav_file))
                    wav_scp.write(f"{identifier}\t{wav_file_path}\n")
                    logging.info(f"寫入 wav.scp: {identifier}\t{wav_file_path}")
        except Exception as e:
            logging.error(f"寫入 wav.scp 失敗於 {wav_scp_path}，錯誤: {e}")

def main():
    analysis_dir = "/jnrle/LDVforDysarthria/TrainingData"

    if not os.path.exists(analysis_dir):
        logging.error(f"分析目錄不存在: {analysis_dir}")
        print(f"分析目錄不存在: {analysis_dir}")
        return

    logging.info("開始重寫 wav.scp 檔案")
    rewrite_wav_scp(analysis_dir)
    logging.info("重寫 wav.scp 檔案完成！")
    print("重寫 wav.scp 檔案完成！")

if __name__ == "__main__":
    main()

重寫 wav.scp 檔案完成！


# 創建raw, speed等資料夾

In [1]:
import os
import shutil

# 定義根目錄路徑
root_dir = '/jnrle/LDVforDysarthria/TrainingData'

# 遍歷根目錄下的每個子資料夾
for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    
    if os.path.isdir(subdir_path):
        # 創建 raw 資料夾
        raw_dir = os.path.join(subdir_path, 'raw')
        os.makedirs(raw_dir, exist_ok=True)
        
        # 移動子資料夾內的所有檔案和資料夾到 raw 資料夾
        for item in os.listdir(subdir_path):
            item_path = os.path.join(subdir_path, item)
            if item_path != raw_dir:
                shutil.move(item_path, raw_dir)
        
        # 創建 speed, perturb, final 資料夾
        for folder_name in ['speed', 'perturb', 'final']:
            os.makedirs(os.path.join(subdir_path, folder_name), exist_ok=True)

# 將wav.scp排序

In [7]:
import os
import re

# 定義根目錄路徑
root_dir = '/jnrle/LDVforDysarthria/TrainingData'

# 遍歷所有子資料夾
for subdir, _, files in os.walk(root_dir):
    if 'wav.scp' in files:
        wav_scp_path = os.path.join(subdir, 'wav.scp')
        
        with open(wav_scp_path, 'r') as file:
            lines = file.readlines()
        
        wang_lines = []
        wenen_lines = []
        
        for line in lines:
            if line.startswith('Wang'):
                match = re.search(r'S1_(\d+)_LDV', line)
                if match:
                    number = int(match.group(1))
                    wang_lines.append((number, line))
            elif line.startswith('Wenen'):
                match = re.search(r'real_(\d+)', line)
                if match:
                    number = int(match.group(1))
                    wenen_lines.append((number, line))
        
        # 排序
        wang_sorted = [line for _, line in sorted(wang_lines, key=lambda x: x[0])]
        wenen_sorted = [line for _, line in sorted(wenen_lines, key=lambda x: x[0])]
        
        # 合併排序後的行
        sorted_lines = wang_sorted + wenen_sorted
        
        # 寫回檔案
        with open(wav_scp_path, 'w') as file:
            file.writelines(sorted_lines)

In [11]:
import os

# 遍歷所有子資料夾
for root, dirs, files in os.walk('/jnrle/LDVforDysarthria/TrainingData'):
    if 'wav.scp' in files and 'utt2spk' in files:
        # 讀取utt2spk建立映射
        utt_map = {}
        with open(os.path.join(root, 'utt2spk'), 'r') as f:
            for line in f:
                utt_id = line.split()[0]
                if utt_id.startswith('Wang'):
                    key = f"Wang_org_S1_{utt_id.split('_')[3]}_LDV"
                    utt_map[key] = utt_id
                elif utt_id.startswith('spk03'):
                    key = f"Wenen_VITS_S1_real_{utt_id.split('_')[-1]}"
                    utt_map[key] = utt_id

        # 讀取並修改wav.scp
        new_lines = []
        with open(os.path.join(root, 'wav.scp'), 'r') as f:
            for line in f:
                parts = line.split(maxsplit=1)
                if len(parts) == 2:
                    old_id = parts[0]
                    if old_id in utt_map:
                        new_lines.append(f"{utt_map[old_id]} {parts[1]}")
                    else:
                        new_lines.append(line)
                else:
                    new_lines.append(line)

        # 寫回修改後的内容
        with open(os.path.join(root, 'wav.scp'), 'w') as f:
            f.writelines(new_lines)

# 生成exp_learning資料夾

In [14]:
import os

# 源目錄和目標目錄
src_dir = '/jnrle/LDVforDysarthria/TrainingData'
dst_dir = '/jnrle/LDVforDysarthria/exp_learning'

# 確保目標目錄存在
os.makedirs(dst_dir, exist_ok=True)

# 獲取源目錄下的一級子目錄並創建對應的空目錄
for item in os.listdir(src_dir):
    src_path = os.path.join(src_dir, item)
    if os.path.isdir(src_path):
        # 創建空目錄
        dst_path = os.path.join(dst_dir, item)
        os.makedirs(dst_path, exist_ok=True)
        
        # 在每個目錄中創建空的info.txt
        with open(os.path.join(dst_path, 'info.txt'), 'w') as f:
            pass  # 創建空文件

In [2]:
import os

# 定義源目錄和要替換的路徑
base_dir = '/work/Jnrle/LDVforDysarthria/TrainingData'
old_prefix = '/jnrle/LDVforDysarthria/TrainingData'
new_prefix = '/work/Jnrle/LDVforDysarthria/TrainingData'

# 遍歷所有子目錄
for root, dirs, files in os.walk(base_dir):
    if 'wav.scp' in files:
        wav_scp_path = os.path.join(root, 'wav.scp')
        with open(wav_scp_path, 'r') as file:
            lines = file.readlines()
        
        # 替換路徑
        new_lines = []
        for line in lines:
            parts = line.strip().split(' ', 1)
            if len(parts) == 2 and parts[1].startswith(old_prefix):
                parts[1] = parts[1].replace(old_prefix, new_prefix, 1)
                new_lines.append(' '.join(parts) + '\n')
            else:
                new_lines.append(line)
        
        # 寫回文件
        with open(wav_scp_path, 'w') as file:
            file.writelines(new_lines)

In [5]:
import os

# 定義源目錄
base_dir = '/work/Jnrle/LDVforDysarthria/TrainingData/Wang_real_addnoise_LDV_caffeteria_n55_LDV_s40'

# 遍歷所有子目錄
for root, dirs, files in os.walk(base_dir):
    if 'wav.scp' in files:
        wav_scp_path = os.path.join(root, 'wav.scp')
        with open(wav_scp_path, 'r') as file:
            lines = file.readlines()
        
        # 替換 /wav/ 為 /raw/wav/
        new_lines = []
        for line in lines:
            parts = line.strip().split(' ', 1)
            if len(parts) == 2 and '/raw/raw/' in parts[1]:
                parts[1] = parts[1].replace('/raw/raw/', '/raw/', 1)
                new_lines.append(' '.join(parts) + '\n')
            else:
                new_lines.append(line)
        
        # 寫回文件
        with open(wav_scp_path, 'w') as file:
            file.writelines(new_lines)

In [10]:
import os

# Define paths
OLD_PATH = '/work/Jnrle/LDVforDysarthria/TrainingData/'
NEW_PATH = '/work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData/'
BASE_DIR = '/work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData/'

def update_wav_scp():
    # Walk through all directories
    for root, dirs, files in os.walk(BASE_DIR):
        if 'wav.scp' in files and 'raw' in root:
            wav_scp_path = os.path.join(root, 'wav.scp')
            print(f"\nProcessing: {wav_scp_path}")
            
            # Read and replace content
            try:
                with open(wav_scp_path, 'r') as file:
                    content = file.read()
                
                # Replace old path with new path
                new_content = content.replace(OLD_PATH, NEW_PATH)
                
                # Write back to file
                with open(wav_scp_path, 'w') as file:
                    file.write(new_content)
                    
                print(f"Updated: {wav_scp_path}")
            except Exception as e:
                print(f"Error processing {wav_scp_path}: {e}")

if __name__ == "__main__":
    print("Starting path update process...")
    update_wav_scp()
    print("\nUpdate completed!")

Starting path update process...

Processing: /work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData/Wang_real_addnoise_Mic_street_n65_Mic_s70/raw/wav.scp
Updated: /work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData/Wang_real_addnoise_Mic_street_n65_Mic_s70/raw/wav.scp

Processing: /work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData/Wang_real_addnoise_LDV_street_n75_LDV_s40/raw/wav.scp
Updated: /work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData/Wang_real_addnoise_LDV_street_n75_LDV_s40/raw/wav.scp

Processing: /work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData/Wang_real_addnoise_LDV_caffeteria_n65_LDV_s60/raw/wav.scp
Updated: /work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData/Wang_real_addnoise_LDV_caffeteria_n65_LDV_s60/raw/wav.scp

Processing: /work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData/Wang_real_addnoise_LDV_caffeteria_n75_LDV_s40/raw/wav.scp
Updated: /work/jerryfat/kaldi-trunk/egs/chiangyihan

In [11]:
import os
import shutil
from pathlib import Path

# 定義常數
BASE_DIR = '/work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData'
REFERENCE_DIR = 'Test_Wang_real_addnoise_LDV_caffeteria_n55_LDV_s40'
KEEP_FILES = {'spk2utt', 'text', 'text_raw', 'utt2spk', 'wav.scp'}
KEEP_DIRS = {'wav'}

def verify_reference():
    """檢查參考目錄結構"""
    ref_path = Path(BASE_DIR) / REFERENCE_DIR
    print(f"\nVerifying reference directory: {REFERENCE_DIR}")
    if ref_path.exists():
        print("Files in reference raw directory:")
        for item in (ref_path / 'raw').iterdir():
            print(f"- {item.name}")
    else:
        print("Reference directory not found!")
        return False
    return True

def clean_directories():
    for subdir in os.listdir(BASE_DIR):
        subdir_path = Path(BASE_DIR) / subdir
        if not subdir_path.is_dir():
            continue
            
        print(f"\nProcessing directory: {subdir}")
        
        # 清理主目錄，只保留 lexicon.txt 和 raw
        for item in subdir_path.iterdir():
            if item.name not in ['lexicon.txt', 'raw']:
                try:
                    if item.is_dir():
                        shutil.rmtree(item)
                    else:
                        item.unlink()
                    print(f"Removed: {item.name}")
                except Exception as e:
                    print(f"Error removing {item.name}: {e}")
        
        # 清理 raw 目錄
        raw_path = subdir_path / 'raw'
        if raw_path.exists():
            for item in raw_path.iterdir():
                # 修正邏輯：只有當檔案/目錄不在保留清單中才刪除
                if (item.name not in KEEP_FILES) and (item.name not in KEEP_DIRS):
                    try:
                        if item.is_dir():
                            shutil.rmtree(item)
                        else:
                            item.unlink()
                        print(f"Removed from raw: {item.name}")
                    except Exception as e:
                        print(f"Error removing {item.name} from raw: {e}")

if __name__ == "__main__":
    print("Starting cleanup process...")
    if verify_reference():
        clean_directories()
        print("\nCleanup completed!")
    else:
        print("Cleanup aborted!")

Starting cleanup process...

Verifying reference directory: Test_Wang_real_addnoise_LDV_caffeteria_n55_LDV_s40
Files in reference raw directory:
- text_raw
- utt2spk
- wav
- text
- spk2utt
- wav.scp

Processing directory: Wang_real_addnoise_Mic_street_n65_Mic_s70
Removed: final

Processing directory: Wang_real_addnoise_LDV_street_n75_LDV_s40
Removed: final

Processing directory: Wang_real_addnoise_LDV_caffeteria_n65_LDV_s60
Removed: final

Processing directory: Wang_real_addnoise_LDV_caffeteria_n75_LDV_s40
Removed: final

Processing directory: Wang_real_addnoise_Mic_n0_Mic_s70
Removed: final

Processing directory: Wang_real_addnoise_Mic_caffeteria_n65_Mic_s40
Removed: final

Processing directory: Wang_real_addnoise_Mic_street_n65_Mic_s55
Removed: final

Processing directory: Wang_real_addnoise_LDV_street_n65_LDV_s55
Removed: final

Processing directory: Wang_real_addnoise_Mic_street_n75_Mic_s70
Removed: final

Processing directory: Wang_real_addnoise_Mic_n0_s60
Removed: final

Processi

In [3]:
pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 36.3 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install openpyxl

import os
import pandas as pd
from pathlib import Path

# 定義基礎目錄
BASE_DIR = '/work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData'

# 確認目錄存在
if not os.path.isdir(BASE_DIR):
    print(f"目錄不存在: {BASE_DIR}")
else:
    # 列出所有子資料夾
    subfolders = [f.name for f in Path(BASE_DIR).iterdir() if f.is_dir()]
    
    # 創建 DataFrame
    df = pd.DataFrame(subfolders, columns=['資料夾名稱'])
    
    # 顯示 DataFrame
    display(df)
    
    # 定義輸出檔案路徑
    output_file = Path(BASE_DIR) / 'TrainingData_Folders.xlsx'
    
    # 存成 Excel
    df.to_excel(output_file, index=False)
    
    print(f"已存成 Excel 檔案: {output_file}")
    


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


,資料夾名稱
0,Wang_real_addnoise_Mic_street_n65_Mic_s70
1,Wang_real_addnoise_LDV_street_n75_LDV_s40
2,Wang_real_addnoise_LDV_caffeteria_n65_LDV_s60
3,Wang_real_addnoise_LDV_caffeteria_n75_LDV_s40
4,Wang_real_addnoise_Mic_n0_Mic_s70
...,...
95,Wang_real_addnoise_Mic_street_n75_Mic_s55
96,Wang_real_addnoise_LDV_street_n75_LDV_s50
97,Wang_real_addnoise_Mic_caffeteria_n75_Mic_s45
98,Wang_real_addnoise_Mic_caffeteria_n65_Mic_s45


已存成 Excel 檔案: /work/jerryfat/kaldi-trunk/egs/chiangyihan/s5/data/TrainingData/TrainingData_Folders.xlsx
